<a href="https://colab.research.google.com/github/CrissRMFI/TA047R-2C2024-GRUPO08/blob/main/TA047R_TP2_GRUPO08_ENTREGA_N4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANÁLISIS EXPLORATORIO

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy import displacy

Cargamos los datos del dataframe

In [ ]:
!wget "https://raw.githubusercontent.com/CrissRMFI/TA047R-2C2024-GRUPO08/refs/heads/main/TP2/test.csv";

--2024-12-01 14:40:23--  https://raw.githubusercontent.com/CrissRMFI/TA047R-2C2024-GRUPO08/refs/heads/main/TP2/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1817926 (1.7M) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]   1.73M  --.-KB/s    in 0.07s   

2024-12-01 14:40:23 (25.1 MB/s) - ‘test.csv’ saved [1817926/1817926]



In [ ]:
!wget "https://raw.githubusercontent.com/CrissRMFI/TA047R-2C2024-GRUPO08/refs/heads/main/TP2/train.csv";

--2024-12-01 14:40:23--  https://raw.githubusercontent.com/CrissRMFI/TA047R-2C2024-GRUPO08/refs/heads/main/TP2/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7301430 (7.0M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   6.96M  --.-KB/s    in 0.09s   

2024-12-01 14:40:24 (74.4 MB/s) - ‘train.csv’ saved [7301430/7301430]



In [ ]:
train_set = pd.read_csv("./train.csv")
test_set = pd.read_csv("./test.csv")

In [ ]:
train_set.head(5)

,id,title,description,project,storypoint
0,5660,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,3
1,9014,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,5
2,4094,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1,5
3,811,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5,2
4,4459,transform processor with script option is broken,Creating the following stream throws exception...,project6,2


In [ ]:
train_set.shape

(7900, 5)

In [ ]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1975 entries, 0 to 1974
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1975 non-null   int64 
 1   title        1975 non-null   object
 2   description  1975 non-null   object
 3   project      1975 non-null   object
dtypes: int64(1), object(3)
memory usage: 61.8+ KB


Verificar si posee datos nulos

In [ ]:
print(train_set.isna().sum() / train_set.shape[0] * 100)

id             0.0
title          0.0
description    0.0
project        0.0
storypoint     0.0
dtype: float64


In [ ]:
print(test_set.isna().sum() / test_set.shape[0] * 100)

id             0.0
title          0.0
description    0.0
project        0.0
dtype: float64


# Pre-Procesamiento

Separamos los datos en variables de entrada y etiquetas

In [ ]:
X_test_text = test_set[['title','description','project']]

In [ ]:
X_train_text = train_set[['title','description','project']]
Y_train = train_set['storypoint']

In [ ]:
X_train_text.head(5)

,title,description,project
0,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8
1,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6
2,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1
3,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5
4,transform processor with script option is broken,Creating the following stream throws exception...,project6


Creamos una funcion que lemmatiza las palabras y nos devuelve las 5 palabras más comunes en un texto y otra que va a agarrar las oraciones de un texto y las va a separar en una lista. La primera la vamos a usar para los titulos y la otra para las descripciones ya que poseen más oraciones.

In [ ]:
nlp = spacy.load("en_core_web_lg")

from collections import Counter
def dejarPalabrasComunes (texto):
  textoTokenizado = nlp(texto)
  palabras = [];
  for token in textoTokenizado:
    if(token.is_alpha and not token.is_stop and not token.is_punct):
      palabras.append(token.lemma_.lower());
  frecuencia = Counter(palabras)
  return frecuencia.most_common(4)

def agarrarOraciones (texto):
  textoTokenizado = nlp(texto);
  oraciones = [];
  for sent in textoTokenizado.sents:
    oracion = [];
    for token in sent:
      if(token.is_alpha):
        oracion.append(token.lemma_.lower())
    oraciones.append(nlp(" ".join(oracion)));
  return oraciones;


In [ ]:
X_train_token = X_train_text.copy()
X_train_token['oraciones descripcion'] = X_train_text['description'].map(lambda x : agarrarOraciones(x))
X_test_token = X_test_text.copy()
X_test_token['oraciones descripcion'] = X_test_text['description'].map(lambda x : agarrarOraciones(x))

In [ ]:
X_train_token['palabras titulo'] = X_train_text['title'].map(lambda x : dejarPalabrasComunes(x))
X_test_token['palabras titulo'] = X_test_text['title'].map(lambda x : dejarPalabrasComunes(x))

In [ ]:
X_train_token.head(10)

,title,description,project,oraciones descripcion,palabras titulo
0,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,"[(when, create, the, default, app, i, encounte...","[(error, 1), (enable, 1), (appcelerator, 1), (..."
1,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,"[(as, a, developer, i, like, to, have, a, main...","[(create, 1), (maintenance, 1), (branch, 1)]"
2,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1,"[(sam, api, use, by, sam, gui)]","[(service, 1), (activity, 1), (monitoring, 1),..."
3,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5,"[(i, notice, this, when, i, be, test, the, uni...","[(work, 1), (rootfs, 1), (read, 1)]"
4,transform processor with script option is broken,Creating the following stream throws exception...,project6,"[(create, the, follow, stream, throw, exceptio...","[(transform, 1), (processor, 1), (script, 1), ..."
5,Service Registry: REST endpoints URL renaming,Endpoint URLs of Service Registry REST service...,project1,"[(endpoint, url, of, service, registry, rest, ...","[(service, 1), (registry, 1), (rest, 1), (endp..."
6,TiStudio: Projects - A pane is missing when cr...,Details: If you try to create the default Tita...,project8,"[(detail, if, you, try, to, create, the, defau...","[(project, 2), (tistudio, 1), (pane, 1), (miss..."
7,Avoid installing test and performance services...,The membership plans are associated with organ...,project8,"[(the, membership, plan, be, associate, with, ...","[(avoid, 1), (instal, 1), (test, 1), (performa..."
8,Add Unit Tests for CC SPI infrastructure,"Test Converter, Configuration, Definition and ...",project6,"[(test, converter, configuration, definition, ...","[(add, 1), (unit, 1), (tests, 1), (cc, 1)]"
9,Agent should not return partial state when a r...,Currently when a user is hitting /state.json o...,project5,"[(currently, when, a, user, be, hit, on, the, ...","[(agent, 1), (return, 1), (partial, 1), (state..."


Creamos un index de palabras que va a funcionar como una bolsa de palabras tanto para el titulo como para la descripción de los storypoints.
Y un index de oraciones comunes, para esto comparamos las oraciones y dependiendo de su similitud con las que ya estan en el index decide si incluirlas o no

In [ ]:
indexOracionesDescripcion = {};
indexPalabrasTitulo = {};

def agregarYDevolverIndex(lista, index, palabraClave):
  indexNuevo = [];
  for elemento in lista:
    if (elemento[0] not in index):
      index[elemento[0]] = str(len(index))+palabraClave
    indexNuevo.append(index[elemento[0]]);
  return indexNuevo;

def agregarYDevolverIndexOracion(lista, index, palabraClave):
  umbral_similitud = 0.75
  indexNuevo = [];
  claves = {key: key for key in index.keys()}
  for elemento in lista:
    similar = False;
    keyElemento = None;
    for key in claves:
      if(elemento.similarity(key) >= umbral_similitud):
          similar = True;
          keyElemento = index[key]
          break;
    if (not similar):
      index[elemento] = str(len(index))+palabraClave
      keyElemento = index[elemento]
    tokenKeyElemento = nlp(keyElemento);
    indexNuevo.append(keyElemento);
  return indexNuevo;

In [ ]:
palabraClaveTitulo = "_T"
palabraClaveDescripcion= "_D"
X_train_index = X_train_token.copy()
X_train_index['index descripcion'] = X_train_token['oraciones descripcion'].map(lambda x : agregarYDevolverIndexOracion(x, indexOracionesDescripcion, palabraClaveDescripcion))

<ipython-input-18-9b767c0cd8bb>:20: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if(elemento.similarity(key) >= umbral_similitud):


In [ ]:
X_test_index = X_test_token.copy()
X_test_index['index descripcion'] = X_test_token['oraciones descripcion'].map(lambda x : agregarYDevolverIndexOracion(x, indexOracionesDescripcion, palabraClaveDescripcion))

<ipython-input-18-9b767c0cd8bb>:20: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if(elemento.similarity(key) >= umbral_similitud):


In [ ]:
X_train_index['index titulo'] = X_train_token['palabras titulo'].map(lambda x : agregarYDevolverIndex(x, indexPalabrasTitulo, palabraClaveTitulo))
X_test_index['index titulo'] = X_test_token['palabras titulo'].map(lambda x : agregarYDevolverIndex(x, indexPalabrasTitulo, palabraClaveTitulo))

In [ ]:
X_train_index.head(10)

,title,description,project,oraciones descripcion,palabras titulo,index descripcion,index titulo
0,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,"[(when, create, the, default, app, i, encounte...","[(error, 1), (enable, 1), (appcelerator, 1), (...","[0_D, 1_D, 2_D]","[0_T, 1_T, 2_T, 3_T]"
1,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,"[(as, a, developer, i, like, to, have, a, main...","[(create, 1), (maintenance, 1), (branch, 1)]",[0_D],"[4_T, 5_T, 6_T]"
2,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1,"[(sam, api, use, by, sam, gui)]","[(service, 1), (activity, 1), (monitoring, 1),...",[3_D],"[3_T, 7_T, 8_T, 9_T]"
3,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5,"[(i, notice, this, when, i, be, test, the, uni...","[(work, 1), (rootfs, 1), (read, 1)]","[0_D, 0_D, 2_D, 4_D, 0_D]","[10_T, 11_T, 12_T]"
4,transform processor with script option is broken,Creating the following stream throws exception...,project6,"[(create, the, follow, stream, throw, exceptio...","[(transform, 1), (processor, 1), (script, 1), ...","[0_D, 5_D, 0_D]","[13_T, 14_T, 15_T, 16_T]"
5,Service Registry: REST endpoints URL renaming,Endpoint URLs of Service Registry REST service...,project1,"[(endpoint, url, of, service, registry, rest, ...","[(service, 1), (registry, 1), (rest, 1), (endp...","[0_D, 0_D, 0_D, 6_D, 2_D, 7_D, 0_D]","[3_T, 17_T, 18_T, 19_T]"
6,TiStudio: Projects - A pane is missing when cr...,Details: If you try to create the default Tita...,project8,"[(detail, if, you, try, to, create, the, defau...","[(project, 2), (tistudio, 1), (pane, 1), (miss...","[0_D, 4_D, 8_D, 0_D, 0_D, 1_D]","[20_T, 21_T, 22_T, 23_T]"
7,Avoid installing test and performance services...,The membership plans are associated with organ...,project8,"[(the, membership, plan, be, associate, with, ...","[(avoid, 1), (instal, 1), (test, 1), (performa...","[0_D, 0_D, 0_D]","[24_T, 25_T, 26_T, 27_T]"
8,Add Unit Tests for CC SPI infrastructure,"Test Converter, Configuration, Definition and ...",project6,"[(test, converter, configuration, definition, ...","[(add, 1), (unit, 1), (tests, 1), (cc, 1)]",[0_D],"[28_T, 29_T, 30_T, 31_T]"
9,Agent should not return partial state when a r...,Currently when a user is hitting /state.json o...,project5,"[(currently, when, a, user, be, hit, on, the, ...","[(agent, 1), (return, 1), (partial, 1), (state...","[0_D, 0_D, 0_D, 2_D, 9_D, 2_D, 10_D, 11_D, 12_...","[32_T, 33_T, 34_T, 35_T]"


In [ ]:
print("Hay "+str(len(indexOracionesDescripcion))+" oraciones diferentes en las descripciones"),
print("Hay "+str(len(indexPalabrasTitulo))+" palabras diferentes comunes en los titulos")

Hay 3292 oraciones diferentes en las descripciones
Hay 4020 palabras diferentes comunes en los titulos


Convertimos las palabras que incluimos en los index a texto, usando de columnas el index asociado a la palabra u oración

In [ ]:
from collections import Counter

def contar(index, palabras):
    counterPalabras = Counter(palabras)
    counterIndex = Counter({key: 0 for key in index.values()})
    counterIndex.update(counterPalabras)
    return counterIndex

In [ ]:
x_train_indices = X_train_index.copy()
x_train_indices['index description count'] = x_train_indices['index descripcion'].map(lambda x :contar(indexOracionesDescripcion, x));
x_train_indices =  x_train_indices.join(pd.json_normalize(x_train_indices['index description count']))

In [ ]:
x_test_indices = X_test_index.copy()
x_test_indices['index description count'] = x_test_indices['index descripcion'].map(lambda x :contar(indexOracionesDescripcion, x));
x_test_indices =  x_test_indices.join(pd.json_normalize(x_test_indices['index description count']))

In [ ]:
x_train_indices['index title count'] = x_train_indices['index titulo'].map(lambda x :contar(indexPalabrasTitulo, x));
x_train_indices =  x_train_indices.join(pd.json_normalize(x_train_indices['index title count']))
x_train_indices = x_train_indices.fillna(0)

In [ ]:
x_test_indices['index title count'] = x_test_indices['index titulo'].map(lambda x :contar(indexPalabrasTitulo, x));
x_test_indices =  x_test_indices.join(pd.json_normalize(x_test_indices['index title count']))
x_test_indices = x_test_indices.fillna(0)

Eliminamos columnas que ya no son necesarias

In [ ]:
x_train_reducido = x_train_indices.drop(["title",	"description", "oraciones descripcion","palabras titulo","index descripcion","index titulo","index description count", "index title count"], axis= 1)
x_test_reducido= x_test_indices.drop(["title",	"description","oraciones descripcion","palabras titulo","index descripcion","index titulo","index description count", "index title count"], axis= 1)

In [ ]:
x_train_reducido.head(2)

,project,0_D,1_D,2_D,3_D,4_D,5_D,6_D,7_D,8_D,...,4010_T,4011_T,4012_T,4013_T,4014_T,4015_T,4016_T,4017_T,4018_T,4019_T
0,project8,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,project6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Nos dimos cuenta que se nos estaba consumiendo la RAM por la cantidad de datasets y la cantidad de columnas que posee, asi que pasamos los dataset a un csv que más adelante va a volver a agarrar para entrenar al modelo

In [ ]:
x_train_reducido.to_csv('x_train_reducido.csv', index=False, header=True)

In [ ]:
x_test_reducido.to_csv('x_test_reducido.csv', index=False, header=True)

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import numpy as np
import pandas as pd
X_train = pd.read_csv("./x_train_reducido.csv")
X_test= pd.read_csv("./x_test_reducido.csv")

In [ ]:
X_train.head(2)

,project,0_D,1_D,2_D,3_D,4_D,5_D,6_D,7_D,8_D,...,4010_T,4011_T,4012_T,4013_T,4014_T,4015_T,4016_T,4017_T,4018_T,4019_T
0,project8,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,project6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Convertimos la variable categoria "project" a una númerica para que pueda ser tomado por el modelo

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
X_train['project'] = label_encoder.fit_transform(X_train['project'])

In [ ]:
X_train

,project,0_D,1_D,2_D,3_D,4_D,5_D,6_D,7_D,8_D,...,4010_T,4011_T,4012_T,4013_T,4014_T,4015_T,4016_T,4017_T,4018_T,4019_T
0,7,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,3,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,2,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7895,6,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7896,4,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7897,0,7,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7898,7,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_test['project'] = label_encoder.fit_transform(X_test['project'])

# Entrenamiento del Modelo y Predicciones

Vamos a utilizar un modelo de ensamble por cascading, es decir, los modelos se entrenan secuencialmente, con las predicciones del modelo anterior como caracteristicas adicional para entrenar al siguiente modelo

In [ ]:
Y_train = pd.read_csv("./train.csv")['storypoint']

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV


parametros = {'criterion':['squared_error'],
              'min_samples_split': list(range(4,21)),
              'min_samples_leaf': list(range(2,20)),
              'ccp_alpha':np.linspace(0,0.05,15),
              'max_depth':list(range(1,21)),
              'max_features': [0.5, 0.6, 0.7, 0.8, 0.9]}
numeroFolds=10
kFold = KFold(n_splits=numeroFolds)
arbolBase =  DecisionTreeRegressor(random_state=13)
scorer = 'neg_root_mean_squared_error'

randomCv= RandomizedSearchCV(estimator=arbolBase,
                             random_state=13,
                             param_distributions = parametros,
                             scoring=scorer,
                             cv=kFold,
                             n_jobs=1,
                              n_iter=50
                             )
modelArbol = randomCv.fit(X_train,Y_train);


In [ ]:
modelArbol.best_params_

{'min_samples_split': 8,
 'min_samples_leaf': 13,
 'max_features': 0.5,
 'max_depth': 16,
 'criterion': 'squared_error',
 'ccp_alpha': 0.0071428571428571435}

In [ ]:
modelArbol.best_score_

-2.676472673079843

In [ ]:
bestModelArbol = modelArbol.best_estimator_
bestModelArbol

DecisionTreeRegressor(ccp_alpha=0.0071428571428571435, max_depth=16,
                      max_features=0.5, min_samples_leaf=13,
                      min_samples_split=8, random_state=13)

In [ ]:
y_test_predArbol = bestModelArbol.predict(X_test)
y_train_predArbol = bestModelArbol.predict(X_train)

X_train_arbol= np.column_stack((X_train, y_train_predArbol))
X_test_arbol= np.column_stack((X_test, y_test_predArbol))

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

numeroFolds = 5;
parametros = {'objective': ['reg:squarederror'],
              'eval_metric': ['rmse'],
              'max_depth': list(range(3, 15)),
              'learning_rate': [0.01, 0.1, 0.2],
              'subsample':  [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              'n_estimators' : [50,100,150,200],
              }
kFold = KFold(n_splits=numeroFolds)
XGBCBase = xgb.XGBRegressor(random_state=13)
scorer = 'neg_root_mean_squared_error'

randomCv = RandomizedSearchCV(estimator=XGBCBase,
                              random_state=13,
                              param_distributions = parametros,
                              scoring=scorer,
                              cv=kFold,
                              n_jobs=1,
                              n_iter=50
                              )
modelXGBC = randomCv.fit(X_train_arbol, Y_train)

In [ ]:
modelXGBC.best_params_

{'subsample': 1.0,
 'objective': 'reg:squarederror',
 'n_estimators': 100,
 'max_depth': 3,
 'learning_rate': 0.2,
 'eval_metric': 'rmse'}

In [ ]:
modelXGBC.best_score_

-2.6095818058738467

In [ ]:
bestModelXGBC = modelXGBC.best_estimator_
bestModelXGBC

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=13, ...)

In [ ]:
y_test_predXGBC = bestModelXGBC.predict(X_test_arbol)
y_train_predXGBC = bestModelXGBC.predict(X_train_arbol)
X_train_XGBC = np.column_stack((X_train_arbol, y_train_predXGBC))
X_test_XGBC = np.column_stack((X_test_arbol, y_test_predXGBC))

In [ ]:
import lightgbm as lgb
from scipy.stats import uniform, randint

parametros = {'boosting_type': ['gbdt', 'dart', 'goss'],
              'num_leaves': [31, 50, 100],
              'learning_rate': [0.01, 0.1, 0.2],
              'min_data_in_leaf': randint(20, 100),
              'feature_fraction': [0.4, 0.5, 0.6, 0.7, 0.8],
              'max_depth':list(range(3, 15)),
              'lambda_l1': uniform(0.0, 1.0),
              'lambda_l2': uniform(0.0, 1.0),
              'n_estimators':  [100,150,200,250]}
numeroFolds=15
kFold = KFold(n_splits=numeroFolds, shuffle=True, random_state=13)
lgbBase =  lgb.LGBMRegressor(random_state=13)
scorer = 'neg_root_mean_squared_error'
randomCv = RandomizedSearchCV(estimator=lgbBase,
                              random_state=13,
                              param_distributions = parametros,
                              scoring=scorer,
                              cv=kFold,
                              n_jobs=1,
                              n_iter=100
                              )
modelLGB = randomCv.fit(X_train_XGBC, Y_train)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

In [ ]:
modelLGB.best_params_

{'boosting_type': 'gbdt',
 'feature_fraction': 0.5,
 'lambda_l1': 0.08190006001834516,
 'lambda_l2': 0.26938469527986364,
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_data_in_leaf': 28,
 'n_estimators': 200,
 'num_leaves': 100}

In [ ]:
modelLGB.best_score_

-2.260627015506139

In [ ]:
modeloFinal= modelLGB.best_estimator_
modeloFinal

LGBMRegressor(feature_fraction=0.5, lambda_l1=0.08190006001834516,
              lambda_l2=0.26938469527986364, max_depth=5, min_data_in_leaf=28,
              n_estimators=200, num_leaves=100, random_state=13)

In [ ]:
Y_train.head(5)

,storypoint
0,3
1,5
2,5
3,2
4,2


Metricas para el set de entreamiento del modelo

In [ ]:
y_train_pred = modeloFinal.predict(X_train_XGBC)
y_train_pred_rounded = np.round(y_train_pred).astype(int)
print(y_train_pred)

[LightGBM] [Warning] min_data_in_leaf is set=28, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=28
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=0.26938469527986364, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.26938469527986364
[LightGBM] [Warning] lambda_l1 is set=0.08190006001834516, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.08190006001834516
[4.56180995 4.90968961 5.08886009 ... 3.09814795 3.85577991 5.70575018]


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(Y_train, y_train_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_train, y_train_pred)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

metricas = {
    'metrica' : ["MSE", "RMSE", "MAE"],
    'valor' :  [mse, rmse, mae]
}
df_metrics = pd.DataFrame(metricas)
print(df_metrics)

  metrica     valor
0     MSE  4.398440
1    RMSE  2.097246
2     MAE  1.488035


Hallamos las predicciones con respecto al set de test y lo pasamos a un archivo csv para poder subirlo a Kaggle

In [ ]:
y_test_pred = modeloFinal.predict(X_test_XGBC)
print(y_test_pred)

[LightGBM] [Warning] min_data_in_leaf is set=28, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=28
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=0.26938469527986364, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.26938469527986364
[LightGBM] [Warning] lambda_l1 is set=0.08190006001834516, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.08190006001834516
[5.54026195 1.94647031 1.82519416 ... 1.89892244 5.52273588 4.29113779]


In [ ]:
y_test_pred_rounded = np.round(y_test_pred).astype(int)

In [ ]:
test_set = pd.read_csv("./test.csv")
results = pd.DataFrame({
    'id': test_set['id'],
    'storypoint': y_test_pred_rounded
})

results.head(15)

results.to_csv('results.csv', index=False, header=True)

Crear el archivo Pickle

In [ ]:
import pickle

model_pkl_file = "modelo Ensamble.pkl"

with open(model_pkl_file, 'wb') as file:
    pickle.dump(modeloFinal, file)